In [1]:
pip install pymorphy2


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk snowballstemmer pymystem3 gensim

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import pymorphy2
import pymystem3
from snowballstemmer import EnglishStemmer
import nltk
nltk.download('punkt')
import json
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv(r'C:\Users\User\Desktop\labs/lenta-ru-news.csv')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


In [4]:
data['tags'].value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
69-я параллель         1
Инновации              1
Name: tags, Length: 94, dtype: int64

Используем Label Encode, удалим ненужные признаки, а также заполним пропуски в таблице


In [8]:
data.isna().any()

url      False
title    False
text      True
topic     True
tags      True
date     False
dtype: bool

In [9]:
data = data.sample(20000)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2006/07/18/costofwar/,Война с Ливаном стоит Израилю полмиллиарда шек...,Цена войны с Ливаном составляет для Израиля 50...,Мир,Все,2006/07/18
1,https://lenta.ru/news/2010/03/12/football/,Первый гол чемпионата России забил японец в до...,В пятницу в первом матче 1 тура 19-го чемпиона...,Спорт,Все,2010/03/12
2,https://lenta.ru/news/2008/03/19/march1/,"""Другая Россия"" проведет акции перед инаугурац...","Лидеры оппозиционной организации ""Другая Росси...",Россия,Все,2008/03/19
3,https://lenta.ru/news/2015/07/06/election/,В ЛНР определили дату местных выборов,Глава самопровозглашенной Луганской народной р...,Бывший СССР,Украина,2015/07/06
4,https://lenta.ru/news/2013/09/18/proton/,Физики впервые измерили слабый заряд протона,Физики впервые провели экспериментальное измер...,Наука и техника,Наука,2013/09/18
...,...,...,...,...,...,...
17784,https://lenta.ru/news/2003/11/01/hohmann/,Депутат Бундестага сравнил евреев-большевиков ...,Депутат немецкого парламента от Христианско-де...,Мир,Все,2003/11/01
17785,https://lenta.ru/news/2017/12/19/mummy/,Мумия пенсионерки семь лет пролежала в квартире,В квартире одной из многоэтажек Металлургическ...,Дом,Квартира,2017/12/19
17786,https://lenta.ru/news/2014/07/02/su24/,Ополченцы из ПЗРК подбили Су-24,"Бомбардировщик Су-24, принадлежащий украинской...",Бывший СССР,Украина,2014/07/02
17787,https://lenta.ru/news/2011/12/30/kolskaya/,Прокуратура выявила нарушения у собственника п...,Дальневосточная транспортная прокуратура выяви...,Россия,Все,2011/12/30


In [10]:
data = data.drop(['url', 'title', 'date', 'topic'], axis='columns')
data

,text,tags
0,Цена войны с Ливаном составляет для Израиля 50...,Все
1,В пятницу в первом матче 1 тура 19-го чемпиона...,Все
2,"Лидеры оппозиционной организации ""Другая Росси...",Все
3,Глава самопровозглашенной Луганской народной р...,Украина
4,Физики впервые провели экспериментальное измер...,Наука
...,...,...
17784,Депутат немецкого парламента от Христианско-де...,Все
17785,В квартире одной из многоэтажек Металлургическ...,Квартира
17786,"Бомбардировщик Су-24, принадлежащий украинской...",Украина
17787,Дальневосточная транспортная прокуратура выяви...,Все


In [11]:
labelencoder = LabelEncoder()
data['tags'] = labelencoder.fit_transform(data['tags'])
data


,text,tags
0,Цена войны с Ливаном составляет для Израиля 50...,10
1,В пятницу в первом матче 1 тура 19-го чемпиона...,10
2,"Лидеры оппозиционной организации ""Другая Росси...",10
3,Глава самопровозглашенной Луганской народной р...,74
4,Физики впервые провели экспериментальное измер...,46
...,...,...
17784,Депутат немецкого парламента от Христианско-де...,10
17785,В квартире одной из многоэтажек Металлургическ...,30
17786,"Бомбардировщик Су-24, принадлежащий украинской...",74
17787,Дальневосточная транспортная прокуратура выяви...,10


In [12]:
for i in range (11412): 
    if (i == 49) or (i == 53):
        continue
    else:
        data = data[data.tags != i]

In [13]:
data['tags'].value_counts()

53    46
49    25
Name: tags, dtype: int64

In [14]:
data.reset_index(drop=True, inplace=True)
data

,text,tags
0,Москва заняла 17-е место в мире по стоимости а...,49
1,Корреспондента немецкой газеты Die Welt Дениза...,53
2,ФГУП «Российская телевизионная и радиовещатель...,49
3,"Японский ресторан Chojiya, существующий более ...",49
4,Французская актриса русского происхождения Мар...,53
...,...,...
66,В Московском регионе приостановлено возведение...,49
67,Роскомнадзор вынес предупреждения четырем инте...,53
68,Спецкора «Комсомольской Правды» Дарью Асламову...,53
69,Министерство культуры РФ сообщило об успешной ...,49


In [15]:
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(input_text):
    tokens = nltk.word_tokenize(input_text)
    normed_tokens = [morph.parse(s)[0].normal_form for s in tokens]
    # исключим также стоп-слова
    normed_tokens = [word for word in normed_tokens if word not in nltk.corpus.stopwords.words("english")]
    # а также знаки препинания
    normed_tokens = [word for word in normed_tokens if word not in punctuation]
    return ' '.join(normed_tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [16]:
%%time

data['text'] = data['text'].apply(lemmatize)
data

Wall time: 14.8 s


,text,tags
0,москва занять 17-й место в мир по стоимость ар...,49
1,корреспондент немецкий газета die welt дениз ю...,53
2,фгуп « российский телевизионный и радиовещател...,49
3,японский ресторан chojiya существующий более 4...,49
4,французский актриса русский происхождение мари...,53
...,...,...
66,в московский регион приостановить возведение п...,49
67,роскомнадзор вынести предупреждение четыре инт...,53
68,спецкор « комсомольский правда » дарья асламов...,53
69,министерство культура рф сообщить о успешный э...,49


In [17]:
data.head(10)

,text,tags
0,москва занять 17-й место в мир по стоимость ар...,49
1,корреспондент немецкий газета die welt дениз ю...,53
2,фгуп « российский телевизионный и радиовещател...,49
3,японский ресторан chojiya существующий более 4...,49
4,французский актриса русский происхождение мари...,53
5,журналист издание vice отправить в космос кося...,53
6,в торонто с верхушка 65-этажный здание отель t...,49
7,губернатор тверской область игорь руденить при...,49
8,значение ставка капитализация на офисный объек...,49
9,власть москва отозвать с аукцион уникальный па...,49


In [18]:
data['text'][12]

'в москва пройти торг по продажа нежилой помещение площадь 4,7 квадратный метр это самый маленький объект предложить потенциальный покупатель в 2018 год говориться в сообщение столичный департамент по конкурентный политика поступить в « дом » в пятница 27 апрель помещение расположить на первый этаж нежилой шестиэтажный здание построить в 2007 год в северо-западный административный округ по адрес новокуркинский шоссе дом 39 здание оборудовать централизовать водопровод и канализация горячий водоснабжение электричество и скрытый проводка как отмечаться в сообщение выставить на продажа объект находиться в район с хороший инфраструктура — рядом функционировать десять салон красота и точка бытовой обслуживание восемь магазин и кафе пять учебный заведение и культурно-спортивный центр а также четыре объект здравоохранение начальный стоимость лот — 148 тысяча рубль приём заявка на участие в аукцион проводиться до 15 май 2018 год торг состояться 23 май ранее департамент по конкурентный политика 

In [19]:
data = data[data['text'].map(len) > 50]

In [20]:
data = data.reset_index(drop=True)
data

,text,tags
0,москва занять 17-й место в мир по стоимость ар...,49
1,корреспондент немецкий газета die welt дениз ю...,53
2,фгуп « российский телевизионный и радиовещател...,49
3,японский ресторан chojiya существующий более 4...,49
4,французский актриса русский происхождение мари...,53
...,...,...
66,в московский регион приостановить возведение п...,49
67,роскомнадзор вынести предупреждение четыре инт...,53
68,спецкор « комсомольский правда » дарья асламов...,53
69,министерство культура рф сообщить о успешный э...,49


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['tags'], test_size=0.12, stratify=data['tags'], random_state=42)

Bag Of Words


In [22]:
X_train.shape

(62,)

In [23]:
for i in X_train:
  if len(i) < 50:
    print (i)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
bof_vect = CountVectorizer()
bof_vect.fit(np.hstack([X_train, X_test]))
bof_train = bof_vect.transform(X_train)
bof_test = bof_vect.transform(X_test)

In [25]:
bof_train.toarray().shape

(62, 3711)

In [26]:
bof_train.toarray(
)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 2, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

TF-IDF

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(np.hstack([X_train, X_test]))
tfidf_train = tfidf_vect.transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [28]:
tfidf_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.06443812, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [29]:
tfidf_train.toarray().shape

(62, 3711)

Word2Vec

In [30]:
%%time

from gensim.models import Word2Vec
X_train_w2v = X_train.apply(str.split)
X_test_w2v = X_test.apply(str.split)
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]), size=30, min_count=10, workers=8)

Wall time: 686 ms


In [31]:
X_train_w2v

46    [свердловский, прокуратура, начало, доследстве...
66    [в, московский, регион, приостановить, возведе...
22    [смоленский, журналистка, сайт, readovka.ru, п...
64    [кандидат, в, нижний, палата, конгресс, респуб...
57    [мвд, украина, пообещать, отпустить, задержать...
                            ...                        
6     [в, торонто, с, верхушка, 65-этажный, здание, ...
53    [сборная, сша, с, 23, медаль, не, оправдать, н...
8     [значение, ставка, капитализация, на, офисный,...
38    [американский, журнал, rolling, stone, выпусти...
61    [американский, газета, washington, post, и, ne...
Name: text, Length: 62, dtype: object

Преобразуем тексты в вектора - возьмем сумму векторов всех слов в тексте

In [32]:
%%time

import numpy as np

def text2vec(text):
    """Усредняем векторы слов"""
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
    return np.sum(vecs, axis=0) / len(vecs)

w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)
w2v_train

Wall time: 112 ms


<timed exec>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


46    [-0.2281454, 0.23144932, 0.2536811, 0.12859178...
66    [-0.20959441, 0.21670671, 0.23522021, 0.120184...
22    [-0.20311232, 0.20839025, 0.22753915, 0.115746...
64    [-0.21761917, 0.22359414, 0.24602912, 0.126383...
57    [-0.20576707, 0.21167573, 0.23306552, 0.119871...
                            ...                        
6     [-0.2060346, 0.21375301, 0.23220837, 0.1196404...
53    [-0.20957626, 0.21863627, 0.23820375, 0.120427...
8     [-0.20253298, 0.20585422, 0.22750026, 0.114134...
38    [-0.20185126, 0.20556667, 0.22920927, 0.114833...
61    [-0.20600381, 0.21049063, 0.23004277, 0.116169...
Name: text, Length: 62, dtype: object

In [33]:
w2v_train

46    [-0.2281454, 0.23144932, 0.2536811, 0.12859178...
66    [-0.20959441, 0.21670671, 0.23522021, 0.120184...
22    [-0.20311232, 0.20839025, 0.22753915, 0.115746...
64    [-0.21761917, 0.22359414, 0.24602912, 0.126383...
57    [-0.20576707, 0.21167573, 0.23306552, 0.119871...
                            ...                        
6     [-0.2060346, 0.21375301, 0.23220837, 0.1196404...
53    [-0.20957626, 0.21863627, 0.23820375, 0.120427...
8     [-0.20253298, 0.20585422, 0.22750026, 0.114134...
38    [-0.20185126, 0.20556667, 0.22920927, 0.114833...
61    [-0.20600381, 0.21049063, 0.23004277, 0.116169...
Name: text, Length: 62, dtype: object

Классификация

In [34]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, precision_recall_curve, roc_auc_score, 
    f1_score, fbeta_score, auc)

def classsifier_quality(a):
    print('Accuracy:', accuracy_score( y_train.tolist(), a))
    print('Precision:', precision_score( y_train.tolist(), a, average='macro'))
    print('Recall:', recall_score( y_train.tolist(), a, average='macro'))
    print('F1:', f1_score( y_train.tolist(), a, average='macro'))
    print('F0.5:', fbeta_score( y_train.tolist(), a, 0.5 ,average='macro'))
    print('F2:', fbeta_score( y_train.tolist(), a, 2, average='macro'))

In [35]:
%%time 

from sklearn.neighbors import KNeighborsClassifier
bknn_clf = KNeighborsClassifier()
bknn_clf.fit(bof_train.toarray(), y_train.tolist())
bknn_labels = bknn_clf.predict(bof_train.toarray())
bknn_predict = bknn_clf.predict_proba(bof_train.toarray())

Wall time: 194 ms


In [36]:
classsifier_quality(bknn_labels)

Accuracy: 0.9516129032258065
Precision: 0.9437012263099219
Recall: 0.9522727272727273
F1: 0.9476793248945148
F0.5: 0.945220193340494
F2: 0.9503599076463398


C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass beta=0.5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass beta=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [37]:
%%time 

from sklearn.neighbors import KNeighborsClassifier
tknn_clf = KNeighborsClassifier()
tknn_clf.fit(tfidf_train.toarray(), y_train.tolist())
tknn_labels = tknn_clf.predict(tfidf_train.toarray())
tknn_predict = tknn_clf.predict_proba(tfidf_train.toarray())

Wall time: 93.9 ms


In [38]:
classsifier_quality(tknn_labels)

Accuracy: 0.9516129032258065
Precision: 0.94
Recall: 0.9625
F1: 0.9486045869024593
F0.5: 0.9428409487268923
F2: 0.9562688109249361


C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass beta=0.5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass beta=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [39]:
%%time

from sklearn.naive_bayes import GaussianNB
tbayes = GaussianNB()
tbayes.fit(tfidf_train.toarray(), y_train.tolist())
tbayes_labels = tbayes.predict(tfidf_train.toarray())
tbayes_predict = tbayes.predict_proba(tfidf_train.toarray())

Wall time: 51 ms


In [40]:
classsifier_quality(tbayes_labels)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1: 1.0
F0.5: 1.0
F2: 1.0


C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass beta=0.5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass beta=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
